In [1]:
import os
import re
import time
from keras import regularizers
import keras
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from keras import backend as K
from keras.models import Model
from keras import initializers
from keras.engine.topology import Layer
from keras.layers import Dense, Input, RepeatVector, Permute, Multiply, Lambda,BatchNormalization
from keras.layers import Embedding, GRU, Bidirectional, TimeDistributed, LSTM, Dropout
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.utils.np_utils import to_categorical
from nltk import tokenize
import tensorflow as tf
from gensim.models import Word2Vec
from keras.preprocessing.sequence import pad_sequences
from konlpy.tag import Kkma, Okt
from sklearn.model_selection import train_test_split
import kss
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sn
import sentencepiece as spm
from keras.callbacks import EarlyStopping
%matplotlib inline

Using TensorFlow backend.
/home/bigdata/.conda/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/bigdata/.conda/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/bigdata/.conda/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/bigdata/.conda/envs/py36/l

### Hierachcial Attention Layer

In [2]:
class Attention(Layer):
    def __init__(self, regularizer = None, **kwargs):
        super(Attention, self).__init__(**kwargs)
        self.regularizer = regularizer
        self.supports_masking = True

    def build(self, input_shape):
        self.context = self.add_weight(name='context', shape=(input_shape[-1], 1),initializer=initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None),regularizer=self.regularizer,trainable=True)
        super(Attention, self).build(input_shape)
    def compute_mask(self, inputs, mask=None):
        return None

    def call(self, x, mask=None):
        attention_in = K.exp(K.squeeze(K.dot(x, self.context), axis=-1))
        attention = attention_in/K.expand_dims(K.sum(attention_in, axis=-1), -1)

        if mask is not None:
        # use only the inputs specified by the mask
        # import pdb; pdb.set_trace()
            attention = attention*K.cast(mask, 'float32')

        weighted_sum = K.batch_dot(K.permute_dimensions(x, [0, 2, 1]), attention)
        return weighted_sum

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]


In [3]:
! nvidia-smi

Wed Aug 12 08:44:27 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 396.37                 Driver Version: 396.37                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:3B:00.0 Off |                    0 |
| N/A   33C    P0    30W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla P100-PCIE...  Off  | 00000000:D8:00.0 Off |                    0 |
| N/A   27C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        

In [35]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

### 뉴스 데이터 불러오기

In [3]:
news_data = pd.read_csv('Article_unity.csv', encoding = 'cp949')

In [4]:
news_data.columns = ['date', 'category', 'publisher','title', 'content']

In [5]:
news_data

,date,category,publisher,title,content
0,20190101,IT과학,전자신문,NIPA AI융합본부 신설 등 신SW산업 육성 조직개편 단행,정보통신산업진흥원 원장 김창용·NIPA 은 AI융합산업본부와 ICT생태계본부 싱크탱...
1,20190101,IT과학,ZDNet Korea,멜론·지니 음원가격 인상...권리자 권익 향상 목적,저작권 징수규정 개정안 적용...신규가입자 대상 지디넷코리아 안희정 기자 멜론과 지...
2,20190101,IT과학,아시아경제,창작자몫 저작권료 5% 오르자 음원서비스 이용료 30% 인상,멜론·벅스·지니 등 일제히 가격 인상… 원가 인상에 따른 것 아시아경제 이민우 기자...
3,20190101,IT과학,서울신문,고든 정의 TECH 2019년 CPU 시장을 보는 세 가지 관전 포인트,서울신문 나우뉴스 올해 서버 시장에 출시될 2세대 64코어 에픽 프로세서를 공개하는...
4,20190101,IT과학,이데일리,포토강성주 본부장 새해 우편물 소통현장 밤샘 방문,이데일리 김현아 기자 강성주 우정사업본부장 오른쪽 이 새해 1월1일 0시부터 우편물...
5,20190101,IT과학,동아일보,인류 최초 지구에서 가장 먼 곳까지 날아간 무인탐사선 ‘뉴호라이즌스’,사진 동아DB 미국항공우주국 NASA 의 무인탐사선 ‘뉴호라이즌스’ New Hori...
6,20190101,IT과학,ZDNet Korea,여기어때 사용자 이메일 주소 노출 사고,개인정보 이용내역 통지 과정서 발생...KISA에 신고 지디넷코리아 백봉삼 기자 위...
7,20190101,IT과학,서울신문,아하 우주 오시리스렉스 소행성 ‘베누’ 500m 앞까지 접근 성공,서울신문 나우뉴스 오시리스 렉스가 베누와 80km 떨어진 거리에서 촬영한 사진 NA...
8,20190101,IT과학,한국경제,2019년 이 기술들을 주목하라…자율주행車가 배송하고 AI가 경리업무 전담,주요 외신·시장조사업체 선정 DNA 활용한 음악 추천 등 개인화 서비스 한 단계 발...
9,20190101,IT과학,한국경제,가격 올린 멜론·지니뮤직 할인 상품으로 고객몰이,저작권료 인상 따른 충격 완화 스트리밍 이용료는 대부분 동결 김주완 기자 음원 서비...


### 타이틀과 기사 본문 합치기

In [6]:
def title_content_combine(title,content):
    return str(title) + ' ' + str(content) 

In [7]:
news_data['x'] = news_data.apply(lambda x : title_content_combine(x.title, x.content), axis = 1)

In [8]:
data = news_data[['x', 'category']]

### 문자 카테고리 숫자화

In [9]:
def category_to_number(y_data):
    category_list = list(set(y_data.category))
    new_y = []
    for i, category in enumerate(y_data.category):
        new_y.append(category_list.index(category))
    ret = pd.DataFrame(new_y)
    ret.columns = ['category']
    return ret,category_list

In [10]:
x_train  = pd.DataFrame(data.x)

In [11]:
y_train = pd.DataFrame(data.category)
y_train.columns = ['category']

In [12]:
y_train,category_list = category_to_number(y_train)
y_train = to_categorical(np.asarray(y_train))

In [13]:
stop_words = []
with open("한국어불용어100.txt", 'r',encoding='utf-8') as f:
    for line in f.read().split('\n'):
        stop_words.append(line.split('\t')[0])

In [15]:
contents = []
texts = []
not_splitted = []
for i, text in enumerate(x_train.x):
    if i % 10000 == 0:
        print(i)
    sentences =kss.split_sentences(text)
    for sen in sentences:
        texts.append(sen)
    contents.append(sentences)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000


In [40]:
with open('spm_news_sw.txt', 'w', encoding='utf-8') as f:
    for sent in texts:
        f.write('{}\n'.format(sent))
    f.close()

In [16]:
sp = spm.SentencePieceProcessor()

In [17]:
sp.load('news_vocab_sw.model')

True

In [31]:
MAX_SENTENCES = 15
MAX_SENTENCE_LENGTH = 100
result = []
X_train = np.zeros((x_train.shape[0], MAX_SENTENCES, MAX_SENTENCE_LENGTH), dtype='int32')
for i, sentences in enumerate(contents):
    for j, sent in enumerate(sentences):
        if j < MAX_SENTENCES:
            result.append(sp.EncodeAsPieces(sent))
            wordTokens = sp.EncodeAsIds(sent)
            sen = sp.EncodeAsPieces(sent)
            for k in range(len(wordTokens)):
                if k< MAX_SENTENCE_LENGTH:
                    if sen[k] in stop_words:
                        X_train[i][j][k] = 0
                    else:
                        X_train[i][j][k] = wordTokens[k]

In [32]:
data.groupby('category').count()


,x
category,
IT과학,69663
경제,67174
사회,67421
생활문화,74368
세계,23530
오피니언,70355
정치,71934


In [33]:
validation_split = 0.2
nb_validation_samples = int(validation_split * X_train.shape[0])

In [34]:
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
X = X_train[indices]
Y = y_train[indices]

In [35]:
nb_test_samples = int(nb_validation_samples * 0.25)

In [36]:
x_train = X[:-nb_validation_samples]
y_train = Y[:-nb_validation_samples]
x_val = X[-nb_validation_samples:-nb_test_samples]
y_val = Y[-nb_validation_samples:-nb_test_samples]
x_test =X[-nb_test_samples:]
y_test = Y[-nb_test_samples:]

In [37]:
len(x_train), len(x_val), len(x_test),len(y_train), len(y_val), len(y_test)

(355556, 66667, 22222, 355556, 66667, 22222)

In [38]:
embedding_dim = 200

### 사용하는 뉴스 데이터를 이용하여 w2v 모델 생성

In [40]:

w2v_model = Word2Vec(result, size=embedding_dim, window=5, min_count=5,  sg=1, iter= 5)

In [ ]:
#w2v_model.save('news_w2v.model')

In [39]:
w2v_model = Word2Vec.load('news_w2v.model')

In [23]:
with open('news_vocab_sw.vocab', encoding='utf-8') as f:
    Vo = [doc.strip().split("\t") for doc in f]
    
word2idx = {w[0]: i for i, w in enumerate(Vo)}

In [24]:
word_index = list(word2idx.keys())
len(word_index)

300000

In [29]:
def get_vector(word, word2vec_model):
    if word in word2vec_model:
        return word2vec_model[word]
    else:
        return None

In [108]:
def get_embedding_matrix(word_index, embedding_dim, w2v_model,word2idx):
    embedding_matrix = np.zeros((len(list(word_index))+1, embedding_dim))
    for i, word in enumerate(list(word2idx.keys())):
        temp = get_vector(word, w2v_model)

        if temp is not None:
            embedding_matrix[i] = temp
    return embedding_matrix

In [109]:
embm = get_embedding_matrix(word_index, embedding_dim, w2v_model,word2idx)

/home/bigdata/.conda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  
/home/bigdata/.conda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


### Attention 모델 생성 (함수화 할 필요 있음)

In [100]:
def create_model(MAX_SENTENCES, MAX_SENTENCE_LENGTH, word_index, embedding_matrix):
    l2_reg = regularizers.l2(1e-8)

    sentence_in = Input(shape=(MAX_SENTENCE_LENGTH,), dtype='int32')
    embedded_word_seq = Embedding(len(word_index) +1,embedding_dim, weights=[embedding_matrix], input_length = MAX_SENTENCE_LENGTH, trainable= True, mask_zero=True, name='word_embeddings',)(sentence_in)
    word_encoder = Bidirectional(GRU(50, return_sequences=True, kernel_regularizer=l2_reg))(embedded_word_seq)
    dense_transform_w = Dense(100, activation='relu', name='dense_transform_w', kernel_regularizer=l2_reg)(word_encoder)
    attention_weighted_sentence = Model(sentence_in, Attention(name='word_attention', regularizer=l2_reg)(dense_transform_w))
    word_attention_model = attention_weighted_sentence


    texts_in = Input(shape=(MAX_SENTENCES, MAX_SENTENCE_LENGTH), dtype='int32')
    attention_weighted_sentences = TimeDistributed(attention_weighted_sentence)(texts_in)
    sentence_encoder = Bidirectional(GRU(50, return_sequences=True, kernel_regularizer=l2_reg))(attention_weighted_sentences)
    dense_transform_s = Dense(100, activation='relu', name='dense_transform_s',kernel_regularizer=l2_reg)(sentence_encoder)
    attention_weighted_text = Attention(name='sentence_attention', regularizer=l2_reg)(dense_transform_s)
    attention_weighted_text1 = Dropout(100)(attention_weighted_text)
    prediction = Dense(y_train.shape[1], activation='softmax')(attention_weighted_text1)
    model = Model(texts_in, prediction)

    model.compile(optimizer=keras.optimizers.Adam(lr=0.001),loss='categorical_crossentropy',metrics=['acc'])
    
    return model, word_attention_model

In [101]:
model1, word_attention_model1 = create_model(MAX_SENTENCES, MAX_SENTENCE_LENGTH, word_index, embedding_matrix)

In [32]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 15, 100)           0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 15, 100)           60085700  
_________________________________________________________________
bidirectional_2 (Bidirection (None, 15, 100)           45300     
_________________________________________________________________
dense_transform_s (Dense)    (None, 15, 100)           10100     
_________________________________________________________________
sentence_attention (Attentio (None, 100)               100       
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 7)                 707       
Total para

In [36]:
es = EarlyStopping(monitor = 'val_loss')


hist = model.fit(x=[x_train],
          y=[y_train],
          batch_size=512,
          epochs=5,
          shuffle=True,
          callbacks=[es],
          validation_data=([x_val], 
                           [y_val]))

Train on 355556 samples, validate on 66667 samples
Epoch 1/5
355556/355556 [==============================] - 397s 1ms/step - loss: 0.5864 - acc: 0.7892 - val_loss: 0.3878 - val_acc: 0.8583
Epoch 2/5
355556/355556 [==============================] - 387s 1ms/step - loss: 0.2916 - acc: 0.8870 - val_loss: 0.3483 - val_acc: 0.8654
Epoch 3/5
122880/355556 [=========>....................] - ETA: 3:59 - loss: 0.1919 - acc: 0.9184

KeyboardInterrupt: 

### 모델 predict  
#### 테스트 셋 결과 확인

In [50]:
prd = model.predict(x_test).argsort()

In [51]:
tst = y_test.argsort()

In [52]:
count = 0
length = len(y_test)
for i in range(length):
    if prd[i][-1] == tst[i][-1]:
        count+=1
count/length

0.8545585455854559

### Attention weight 시각화 코드

In [53]:
def normalize(text):
    reviews = []
    texts = []
    return kss.split_sentences(text) 

def encode_input(reviews):
    
    
    
    ret = np.zeros((MAX_SENTENCES, MAX_SENTENCE_LENGTH), dtype='int32')
    for j, sent in enumerate(reviews):
        if j < MAX_SENTENCES:
            wordTokens = sp.EncodeAsIds(sent)
            for k in range(len(wordTokens)):
                if k< MAX_SENTENCE_LENGTH:
                    ret[j][k] = wordTokens[k]
            
    return ret

In [95]:
def get_activation_maps(data, num, word_attention_model):
    normalized_text = normalize(data.x[num])
    encoded_text = encode_input(normalized_text)

    hidden_word_encoding_out = Model(inputs=word_attention_model.input,outputs=word_attention_model.get_layer('dense_transform_w').output)
    hidden_word_encodings = hidden_word_encoding_out.predict(encoded_text)
    word_context = word_attention_model.get_layer('word_attention').get_weights()[0]
    u_wattention = encoded_text*np.exp(np.squeeze(np.dot(hidden_word_encodings, word_context)))





    nopad_encoded_text = encoded_text[-len(text):]
    nopad_encoded_text = [list(filter(lambda x: x > 0, sentence)) for sentence in nopad_encoded_text]


    reconstructed_texts = [[sp.DecodeIds([int(i)]) for i in sentence] for sentence in nopad_encoded_text]
    nopad_wattention = u_wattention[:len(normalized_text)]
    nopad_wattention = nopad_wattention/np.expand_dims(np.sum(nopad_wattention, -1), -1)
    nopad_wattention = np.array([attention_seq[:len(sentence)] for attention_seq, sentence in zip(nopad_wattention, nopad_encoded_text)])
    word_activation_maps = []


    for i, att in enumerate(nopad_wattention):
        word_activation_maps.append(list(zip(reconstructed_texts[i], att)))

    hidden_sentence_encoding_out = Model(inputs=model.input, outputs=model.get_layer('dense_transform_s').output)
    hidden_sentence_encodings = np.squeeze(hidden_sentence_encoding_out.predict(np.expand_dims(encoded_text, 0)), 0)
    sentence_context = model.get_layer('sentence_attention').get_weights()[0]
    u_sattention = np.exp(np.squeeze(np.dot(hidden_sentence_encodings, sentence_context), -1))
    nopad_sattention = u_sattention[-len(normalized_text):]

    nopad_sattention = nopad_sattention/np.expand_dims(np.sum(nopad_sattention, -1), -1)

    return list(zip(word_activation_maps, nopad_sattention))


결과 체크를 위해 랜덤 데이터 추출

In [81]:
import random
rnd = random.randint(0, data.shape[0])
rnd

108967

In [98]:
am = get_activation_maps(data, rnd,  word_attention_model)
am

[([('ET', 0.09583434347389568),
   ('투자뉴스', 0.2824348374771326),
   ('SK', 0.0021115712670261304),
   ('네트웍스', 0.007248049345483853),
   ('2019', 4.893261000564e-06),
   ('년', 5.275965419920034e-07),
   ('성장사업', 0.061779558575751965),
   ('...', 4.568190230167512e-06),
   ('BUY', 0.013530385019845268),
   ('삼성증권', 0.10266466793558415),
   ('삼성증권', 0.033584444657849225),
   ('에서', 4.643414737150925e-06),
   ('10', 1.4525018235353817e-05),
   ('일', 3.72589270552257e-07),
   ('SK', 2.0335640582279926e-05),
   ('네트웍스', 0.0057101018705161604),
   ('0017', 0.3459419458457621),
   ('40', 0.0008943208578242334),
   ('에', 1.134962544585712e-06),
   ('대해', 1.1294560204287656e-06),
   ('2019', 2.911720089722113e-06),
   ('년', 1.9639532150245513e-07),
   ('성장사업', 0.01025200757127669),
   ('의', 1.6711197104609935e-08),
   ('긍정적', 2.220454183749732e-05),
   ('변화에', 3.001408473483319e-05),
   ('주목', 0.00018370824549264535),
   ('라며', 5.695455758057189e-05),
   ('투자의견', 0.0010581846334942837),
   ('을'

In [83]:
activation_map

[([('ET', 0.09583434347389568),
   ('투자뉴스', 0.2824348374771326),
   ('SK', 0.0021115712670261304),
   ('네트웍스', 0.007248049345483853),
   ('2019', 4.893261000564e-06),
   ('년', 5.275965419920034e-07),
   ('성장사업', 0.061779558575751965),
   ('...', 4.568190230167512e-06),
   ('BUY', 0.013530385019845268),
   ('삼성증권', 0.10266466793558415),
   ('삼성증권', 0.033584444657849225),
   ('에서', 4.643414737150925e-06),
   ('10', 1.4525018235353817e-05),
   ('일', 3.72589270552257e-07),
   ('SK', 2.0335640582279926e-05),
   ('네트웍스', 0.0057101018705161604),
   ('0017', 0.3459419458457621),
   ('40', 0.0008943208578242334),
   ('에', 1.134962544585712e-06),
   ('대해', 1.1294560204287656e-06),
   ('2019', 2.911720089722113e-06),
   ('년', 1.9639532150245513e-07),
   ('성장사업', 0.01025200757127669),
   ('의', 1.6711197104609935e-08),
   ('긍정적', 2.220454183749732e-05),
   ('변화에', 3.001408473483319e-05),
   ('주목', 0.00018370824549264535),
   ('라며', 5.695455758057189e-05),
   ('투자의견', 0.0010581846334942837),
   ('을'

In [84]:
data.x[rnd]

'ET투자뉴스SK네트웍스 2019년 성장사업… BUY삼성증권 삼성증권에서 10일 SK네트웍스 001740 에 대해 2019년 성장사업의 긍정적 변화에 주목 라며 투자의견을 BUY 로 제시하였고 아울러 목표주가로는 6 400원을 내놓았다. 삼성증권 백재승 김슬 애널리스트가 동종목에 대하여 이번에 제시한 BUY 의견은 삼성증권의 직전 매매의견에서 큰 변화없이 그대로 유지되는 것이고 작년 연말의 시장컨센서스에 비추어 보면 매수 의견이 계속 유지되고 있는 상태이다. 그리고 최근 분기내 발표된 전체 증권사 리포트의 컨센서스와 비교를 해볼 경우에 오늘 발표된 투자의견은 전체의견에 수렴하고 있어 이번 의견은 시장의 평균적인 기대감이 객관적으로 표현된 것으로 분석되고 있다. 목표주가의 추이를 살펴보면 정체구간을 한동안 보낸 후에 최근 2회 연속 목표가가 높아졌는데 이번에 조정된 상승폭은 이전보다 다소 둔화된 모습이다. 또한 전일 종가 기준으로 볼때 동종목의 현주가는 이번에 제시된 목표가 대비 11.5%의 저평가 요인이 존재한다는 해석이 제시되고 있다.'

### sentence attention이 가장 높았던 문장 추출

### 가장 attention weight이 높았던 문장 중 word attention weight이 높았던 단어 추출

In [103]:
def extract_sentence_and_word(activation_map, words_num, normalized_text):
    max_sentence_weight = 0
    max_sentence_index = -1
    for i, sentence in enumerate(activation_map):
        if sentence[1] > max_sentence_weight:
            max_sentence_weight = sentence[1]
            max_sentence_index= i
    mx_sent = activation_map[max_sentence_index][0]
    mx_sent.sort(key= lambda x: x[1],reverse=True)
    return normalized_text[max_sentence_index], mx_sent[:words_num]

In [104]:
extract_sentence_and_word(activation_map, 3, normalized_text)

('ET투자뉴스SK네트웍스 2019년 성장사업… BUY삼성증권 삼성증권에서 10일 SK네트웍스 001740 에 대해 2019년 성장사업의 긍정적 변화에 주목 라며 투자의견을 BUY 로 제시하였고 아울러 목표주가로는 6 400원을 내놓았다.',
 [('0017', 0.3459419458457621),
  ('투자뉴스', 0.2824348374771326),
  ('삼성증권', 0.10266466793558415)])

### 모델 예측 카테고리 값 확인

In [90]:
ans = model.predict(np.reshape(encoded_text, (1,)+encoded_text.shape))

In [110]:
print("predicted: {}\n  answer:{}".format(category_list[ans.argsort()[0][-1]],category_list[category_list.index(data.category[rnd])]))

predicted: 경제
  answer:경제


In [ ]:
import pickle
with open('catelist.pickle', 'wb') as f:
    pickle.dump(category_list, f)